## Prefect

In [45]:
from prefect import task, Flow
from prefect.tasks.secrets import PrefectSecret
from prefect.tasks.postgres import PostgresExecute, PostgresFetch
import psycopg2 as pg
import requests
import pandas as pd
import time
import yaml
import requests
import datetime

In [46]:
with open("../config.yml", "r") as ymlfile:
    cfg = yaml.load(ymlfile)
api_key = cfg['riotgamesapi']['key']
username = cfg['postgresql']['user']
password = cfg['postgresql']['password']
database = 'teamfighttactics'

/var/folders/8g/67y55mms29zf6yt6q92_3hsr0000gn/T/ipykernel_86021/2003926877.py:2: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  cfg = yaml.load(ymlfile)


In [47]:
query_header = {
        "X-Riot-Token": api_key
}
challenger_url = 'https://euw1.api.riotgames.com/tft/league/v1/challenger'
challenger_response = requests.get(url = challenger_url, headers=query_header).json()

In [48]:
@task
def get_challenger_users(username, password, db_name, api_key):
    """A task that requires credentials to access something. Passing the
    credentials in as an argument allows you to change how/where the
    credentials are loaded (though we recommend using `PrefectSecret` tasks to
    load them."""

    query_header = {"X-Riot-Token": api_key}

    challenger_url = "https://euw1.api.riotgames.com/tft/league/v1/challenger"
    challenger_response = requests.get(url=challenger_url, headers=query_header).json()
    time.sleep(2)

    for i in challenger_response["entries"]:
        summonerId = i["summonerId"]
        summonerName = i["summonerName"]
        leaguePoints = i["leaguePoints"]

        puuid_url = (
            f"https://euw1.api.riotgames.com/tft/summoner/v1/summoners/{summonerId}"
        )
        puuid_response = requests.get(url=puuid_url, headers=query_header).json()
        time.sleep(2)
        puuid = str(puuid_response["puuid"])

        user_query = """
            INSERT INTO public.users(
                id, summonername, summonerid, rankedleague, puuid, leaguepoints, region)
                VALUES (DEFAULT, %s, %s, 'CHALLENGER', %s, %s , 'EUW1')
                """

        query_data = (
            summonerName,
            summonerId,
            puuid,
            leaguePoints,
        )

        PostgresExecute(
            db_name=db_name,
            user=username,
            host="127.0.0.1",
            port=5432,
            query=user_query,
            data=query_data,
            commit=True,
        ).run(password=password)

In [49]:
with Flow("Hello") as flow:
    username = PrefectSecret("USERNAME")
    password = PrefectSecret("PASSWORD")
    dbname= 'teamfighttactics' 
    get_challenger_users(username = username, password = password,db_name = dbname, api_key = api_key )

flow.run()

[2021-10-29 23:08:46+0100] INFO - prefect.FlowRunner | Beginning Flow run for 'Hello'
[2021-10-29 23:08:46+0100] INFO - prefect.TaskRunner | Task 'PASSWORD': Starting task run...
[2021-10-29 23:08:46+0100] INFO - prefect.TaskRunner | Task 'PASSWORD': Finished task run for task with final state: 'Success'
[2021-10-29 23:08:46+0100] INFO - prefect.TaskRunner | Task 'USERNAME': Starting task run...
[2021-10-29 23:08:46+0100] INFO - prefect.TaskRunner | Task 'USERNAME': Finished task run for task with final state: 'Success'
[2021-10-29 23:08:46+0100] INFO - prefect.TaskRunner | Task 'get_challenger_users': Starting task run...
[2021-10-29 23:16:17+0100] INFO - prefect.TaskRunner | Task 'get_challenger_users': Finished task run for task with final state: 'Success'
[2021-10-29 23:16:17+0100] INFO - prefect.FlowRunner | Flow run SUCCESS: all reference tasks succeeded


<Success: "All reference tasks succeeded.">

In [50]:
@task
def get_match_history(username, password, db_name, api_key):
    """A task that requires credentials to access something. Passing the
    credentials in as an argument allows you to change how/where the
    credentials are loaded (though we recommend using `PrefectSecret` tasks to
    load them."""

    riot_header = {"X-Riot-Token": api_key}

    args = {
        "db_name": db_name,
        "user": username,
        "host": "127.0.0.1",
        "port": 5432,
        "commit": True,
    }

    user_query = "SELECT puuid FROM users LIMIT 1"

    all_puuids = PostgresFetch(query=user_query, data=None, fetch="all", **args).run(
        password=password
    )

    for puuid in all_puuids:
        puuid = puuid[0]
        match_id_url = f"https://europe.api.riotgames.com/tft/match/v1/matches/by-puuid/{puuid}/ids?count=10"
        recent_match_ids = requests.get(url=match_id_url, headers=riot_header).json()
        time.sleep(1.6)

        match_id_query = "SELECT matchid FROM matchinfo WHERE matchid = ANY(%s);"
        match_id_data = (recent_match_ids,)
        existing_ids = PostgresFetch(
            query=match_id_query, data=match_id_data, fetch="all", **args
        ).run(password=password)

        existing_ids = [i[0] for i in existing_ids]
        new_matches = set(recent_match_ids).symmetric_difference(set(existing_ids))
        new_matches = list(new_matches)
        print(existing_ids)
        print(new_matches)
        for match_id in new_matches:
            match_detail_url = (
                f"https://europe.api.riotgames.com/tft/match/v1/matches/{match_id}"
            )
            time.sleep(1.6)

            match_detail = requests.get(match_detail_url, headers=riot_header).json()

            for participant in match_detail["info"]["participants"]:

                participant_puuid = participant['puuid']

                puuid_query = """
                    SELECT id FROM users WHERE puuid = %s
                    """

                puuid_data = (
                    participant_puuid,
                )

                user_id = PostgresFetch(query=puuid_query, data=puuid_data, fetch="one", **args).run(
                    password=password
                )

                match_query = """
                    INSERT INTO matchinfo VALUES (DEFAULT,%s,%s,%s)
                    """

                match_data = (
                    match_id,
                    user_id,
                    participant["placement"],
                )

                PostgresExecute(query=match_query, data=match_data, **args).run(
                    password=password
                )

                participant_query = """
                    SELECT id FROM matchinfo WHERE userid = %s and matchid = %s
                    """

                participant_data = (user_id, match_id,)

                match_data_id = PostgresFetch(
                    query=participant_query, data=participant_data, fetch="one", **args
                ).run(password=password)

                for units in participant['units']:
                    character_id = units['character_id']
                    tier = units['tier']
                    items = units['items']
                    item1 =  items[0] if len(items)>0 else None
                    item2 = items[1] if len(items)>1 else None
                    item3 = items[1] if len(items)>2 else None

                unit_query = """
                    INSERT INTO matchunits VALUES (%s,%s,%s,%s,%s,%s)
                    """
                unit_data = (match_data_id, character_id, tier, item1, item2, item3,)

                PostgresExecute(query=unit_query, data=unit_data, **args).run(
                    password=password
                )

In [51]:
with Flow("Get Latest Matches") as flow:
    dbname = "teamfighttactics"
    get_match_history(
        username=username, password=password, db_name=dbname, api_key=api_key
    )

flow.run()

[2021-10-29 23:16:17+0100] INFO - prefect.FlowRunner | Beginning Flow run for 'Get Latest Matches'
[2021-10-29 23:16:17+0100] INFO - prefect.TaskRunner | Task 'PASSWORD': Starting task run...
[2021-10-29 23:16:17+0100] INFO - prefect.TaskRunner | Task 'PASSWORD': Finished task run for task with final state: 'Success'
[2021-10-29 23:16:17+0100] INFO - prefect.TaskRunner | Task 'USERNAME': Starting task run...
[2021-10-29 23:16:17+0100] INFO - prefect.TaskRunner | Task 'USERNAME': Finished task run for task with final state: 'Success'
[2021-10-29 23:16:17+0100] INFO - prefect.TaskRunner | Task 'get_match_history': Starting task run...
[]
['EUW1_5522333968', 'EUW1_5519233328', 'EUW1_5521008996', 'EUW1_5528067029', 'EUW1_5525389240', 'EUW1_5524323777', 'EUW1_5526869212', 'EUW1_5514902434', 'EUW1_5512947437', 'EUW1_5528696425']
[2021-10-29 23:16:46+0100] INFO - prefect.TaskRunner | Task 'get_match_history': Finished task run for task with final state: 'Success'
[2021-10-29 23:16:46+0100] IN

<Success: "All reference tasks succeeded.">